In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import gmaps
import os

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Import weather data from Part 1
csv_file = os.path.join('..', 'WeatherPy', 'output_data', 'cities.csv')
city_df = pd.read_csv(csv_file)
city_df

,Unnamed: 0,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,Nizwá,0,OM,1604282371,32,22.93,57.53,59.31,2.35
1,1,Dzhebariki-Khaya,0,RU,1604282372,83,62.22,135.80,-4.76,3.27
2,2,Miles City,1,US,1604282373,39,46.41,-105.84,50.00,5.82
3,3,Chai Badan,74,TH,1604282374,77,15.20,101.13,79.36,10.31
4,4,Pak Phanang,40,TH,1604282376,94,8.35,100.20,80.60,4.70
...,...,...,...,...,...,...,...,...,...,...
586,586,East Wenatchee Bench,1,US,1604283122,53,47.43,-120.28,50.00,5.82
587,587,San Vicente,0,AR,1604283123,66,-26.62,-54.13,59.59,4.41
588,588,Ostrovnoy,100,RU,1604283125,92,68.05,39.51,39.00,23.42
589,589,College,20,US,1604283073,77,64.86,-147.80,-0.99,1.63
